In [1]:
import random

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import pickle
import sklearn.metrics as metrics

### functions

In [2]:
def edge_features(node_emb_1, node_emb_2, operator):
    node_emb_1 = np.asfarray(node_emb_1,float)
    node_emb_2 = np.asfarray(node_emb_2, float)
    # combine two nodes' embeddings with specificed operator
    if operator == 'Average':
        edge = [((x + y) / 2.0) for x,y in zip(node_emb_1, node_emb_2)]
    elif operator == 'Hadamard':
        edge = [(x * y) for x,y in zip(node_emb_1, node_emb_2)]
    elif operator == 'Weighted-L1':
        edge = [abs(x - y) for x,y in zip(node_emb_1, node_emb_2)]
    elif operator == 'Weighted-L2':
        edge = [abs(x - y)**2 for x,y in zip(node_emb_1, node_emb_2)]
    elif operator == 'Concat':
        edge = np.concatenate((node_emb_1, node_emb_2), axis=None) 
    else:
        print("Generate edge features: Operator not supported")
        print("Use default operator: Weighted-L1")
        edge = [abs(x - y) for x,y in zip(node_emb_1, node_emb_2)]
        
    return edge
def generate_edge_features(edge_list, node_embeddings, operator):
    edge_features_mtx = []
    
    # generate features for each edge in the list
    for node_index_1, node_index_2 in edge_list:
        node_emb_1 = node_embeddings[node_index_1-1]
        node_emb_2 = node_embeddings[node_index_2-1]
        
        edge_features_mtx.append(edge_features(node_emb_1, node_emb_2, operator))
        
    return edge_features_mtx

def generate_train_set(graph_train, num_edge_sample, node_embeddings, edge_operator,):
    edge_list = list(graph_train.edges)
    num_nodes = graph_train.number_of_nodes()
    
    train_edges = []
    train_edges_labels = [1] * num_edge_sample + [0] * num_edge_sample
    
    random.seed(0)
    
    # sample edges with label 1 (true edges)
    for edge_num in range(num_edge_sample):
        rand_index = random.randint(0, len(edge_list) - 1)
        
        #train_edges.append(tuple(edge_list[rand_index]))
        train_edges.append(edge_list[rand_index])
    non_edge_num = 0
    
    # sample edges with label 0 (non-exist edges)
    while(non_edge_num < num_edge_sample):
        rand_nodes = tuple(np.random.randint(low=0,high=num_nodes, size=2))
        
        if rand_nodes not in edge_list:
            train_edges.append(rand_nodes)
            non_edge_num += 1

    train_edges_features_mtx = generate_edge_features(train_edges, node_embeddings, edge_operator)
            
    return train_edges, train_edges_features_mtx, train_edges_labels

def generate_test_set(graph_test, node_embeddings, edge_operator):
    edge_list = graph_test.edges
    nodes_with_edge = set()
    
    for edge in edge_list:
        nodes_with_edge.add(edge[0])
        nodes_with_edge.add(edge[1])
    
    num_nodes = graph_test.number_of_nodes()
    
    test_edges = []
    test_edges_labels = []
    
    num_edge_sample = len(edge_list)
    non_edge_num = 0 
    # sample edges with label 0 (non-exist edges)
    
    while(non_edge_num < num_edge_sample):
        rand_nodes = tuple(np.random.randint(low=0,high=num_nodes, size=2))
        
        if rand_nodes not in edge_list:
            test_edges.append(rand_nodes)
            test_edges_labels.append(0)
            non_edge_num += 1
        
    for edge in edge_list:
        test_edges.append(edge)
        test_edges_labels.append(1)
    
    test_edges_features_mtx = generate_edge_features(test_edges, node_embeddings, edge_operator)
    
    return test_edges, test_edges_features_mtx, test_edges_labels

def build_clf(feature_mtx, response_vec):
   
    logistic_regression_model = LogisticRegression(random_state = 0,max_iter=5000,solver='liblinear',verbose=1,tol=1e-6)
    binary_clf = logistic_regression_model.fit(feature_mtx, response_vec)
    
    return binary_clf

def pred_links(feature_mtx, LR_clf):
    predict_edges_labels = LR_clf.predict(feature_mtx)
    
    return predict_edges_labels

def precision_recall(predict_labels, true_labels):
    true_positive  = false_positive = 0
    true_negative =  false_negative = 0
    
    for p_label, true_label in zip(predict_labels, true_labels):
        
        #print(p_label,true_label)
        if p_label == true_label and true_label == 1:
            true_positive += 1
        elif p_label == true_label and true_label == 0:
            true_negative += 1
        elif p_label != true_label and true_label == 1:
            false_negative += 1
        elif p_label != true_label and true_label == 0:
            false_positive += 1

    print("TP: ", true_positive)
    print("TN: ", true_negative)
    print("FP: ", false_positive)
    print("FN: ", false_negative)
    
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    if (precision + recall) != 0.0:
        f1 = 2 * (precision * recall) / (precision + recall)
        print("F1: {}".format(f1))
    else:
        print("F1: Divide-by-zero")
    
    cm = metrics.confusion_matrix(true_labels, predict_labels)
    print(cm)
    print(metrics.classification_report(true_labels, predict_labels))
    map = metrics.average_precision_score(true_labels, predict_labels)
    print('Mean Average Precision: {}'.format(map))
    fpr, tpr, thresholds = metrics.roc_curve(true_labels, predict_labels)
    roc_auc = metrics.auc(fpr, tpr)
    print('Area Under ROC Curve: {}'.format(roc_auc))
    
    return precision, recall

### Amazon-Food - Monthly
### load data

In [3]:
# load the training and testing graph
with open('../data/amazon_food/amazon_food_monthly_dir.pkl', 'rb') as file:
    graphs = pickle.load(file)
    
graphs = graphs[-20:]
graph_train = graphs[-2]
graph_test = graphs[-1]

# parameters
num_edge_sample = 400

In [4]:
num_snapshots = len(graphs)
num_snapshots

20

In [5]:
emb_list = []
for i in range(num_snapshots):
    file = '../data/amazon_food/N2V_Embeddings/amazon_month/amazon_month_' + str(i) + '.npy'
    node_embedding = np.load(file)
    emb_list.append(node_embedding)
print(len(emb_list))

20


### independent / no combination

In [6]:
node_embeddings_training = emb_list[-3]
node_embeddings_testing = emb_list[-2]

In [7]:
for edge_operator in ['Average','Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  1288
TN:  1541
FP:  1244
FN:  1497
F1: 0.48448373142749673
[[1541 1244]
 [1497 1288]]
              precision    recall  f1-score   support

           0       0.51      0.55      0.53      2785
           1       0.51      0.46      0.48      2785

   micro avg       0.51      0.51      0.51      5570
   macro avg       0.51      0.51      0.51      5570
weighted avg       0.51      0.51      0.51      5570

Mean Average Precision: 0.5040183674106091
Area Under ROC Curve: 0.5078994614003591
Precision:  0.5086887835703001
Recall:  0.46247755834829446
[LibLinear]Edge Operator: Hadamard
TP:  1379
TN:  1366
FP:  1419
FN:  1406
F1: 0.4939996417696579
[[1366 1419]
 [1406 1379]]
              precision    recall  f1-score   support

           0       0.49      0.49      0.49      2785
           1       0.49      0.50      0.49      2785

   micro avg       0.49      0.49      0.49      5570
   macro avg       0.49      0.49      0.49      5570
weighte

### sum

In [8]:
for i in range(len(emb_list)):
    emb_list[i] = np.asfarray(emb_list[i],float)
node_embeddings_training = np.sum(np.asarray(emb_list[0:-2]),axis=0)
node_embeddings_testing = np.sum(np.asarray(emb_list[0:-1]),axis=0)

In [9]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  1294
TN:  1549
FP:  1236
FN:  1491
F1: 0.4869238005644403
[[1549 1236]
 [1491 1294]]
              precision    recall  f1-score   support

           0       0.51      0.56      0.53      2785
           1       0.51      0.46      0.49      2785

   micro avg       0.51      0.51      0.51      5570
   macro avg       0.51      0.51      0.51      5570
weighted avg       0.51      0.51      0.51      5570

Mean Average Precision: 0.5053258208499798
Area Under ROC Curve: 0.5104129263913824
Precision:  0.5114624505928854
Recall:  0.46463195691202874
[LibLinear]Edge Operator: Hadamard
TP:  1305
TN:  1466
FP:  1319
FN:  1480
F1: 0.4825291181364393
[[1466 1319]
 [1480 1305]]
              precision    recall  f1-score   support

           0       0.50      0.53      0.51      2785
           1       0.50      0.47      0.48      2785

   micro avg       0.50      0.50      0.50      5570
   macro avg       0.50      0.50      0.50      5570
weighted

### expdecay

In [16]:
exps = [np.exp(-i * 0.3) for i in range(1,21)]
node_embeddings_training = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-2],exps[:-1]):
    node_embeddings_training += e * c
print(len(emb_list))
print(len(exps))

20
20


In [17]:
node_embeddings_testing = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-1],exps[:-1]):
    node_embeddings_testing += e * c 

In [18]:
for edge_operator in [ 'Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  1278
TN:  1525
FP:  1260
FN:  1507
F1: 0.4801803494270148
[[1525 1260]
 [1507 1278]]
              precision    recall  f1-score   support

           0       0.50      0.55      0.52      2785
           1       0.50      0.46      0.48      2785

   micro avg       0.50      0.50      0.50      5570
   macro avg       0.50      0.50      0.50      5570
weighted avg       0.50      0.50      0.50      5570

Mean Average Precision: 0.5016272584896291
Area Under ROC Curve: 0.5032315978456015
Precision:  0.5035460992907801
Recall:  0.45888689407540395
[LibLinear]Edge Operator: Hadamard
TP:  1344
TN:  1430
FP:  1355
FN:  1441
F1: 0.49015317286652077
[[1430 1355]
 [1441 1344]]
              precision    recall  f1-score   support

           0       0.50      0.51      0.51      2785
           1       0.50      0.48      0.49      2785

   micro avg       0.50      0.50      0.50      5570
   macro avg       0.50      0.50      0.50      5570
weighte

### Amazon-Food - Weekly
### load data

In [19]:
# load the training and testing graph
with open('../data/amazon_food/amazon_food_weekly_dir.pkl', 'rb') as file:
    graphs = pickle.load(file)
    
graphs = graphs[-20:]
graph_train = graphs[-2]
graph_test = graphs[-1]

# parameters
num_edge_sample = 400

In [20]:
num_snapshots = len(graphs)
num_snapshots

20

In [22]:
emb_list = []
for i in range(num_snapshots):
    file = '../data/amazon_food/N2V_Embeddings/amazon_week/amazon_week_' + str(i) + '.npy'
    node_embedding = np.load(file)
    emb_list.append(node_embedding)
print(len(emb_list))

20


### independent / no combination

In [23]:
node_embeddings_training = emb_list[-3]
node_embeddings_testing = emb_list[-2]

In [24]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  582
TN:  744
FP:  534
FN:  696
F1: 0.4862155388471178
[[744 534]
 [696 582]]
              precision    recall  f1-score   support

           0       0.52      0.58      0.55      1278
           1       0.52      0.46      0.49      1278

   micro avg       0.52      0.52      0.52      2556
   macro avg       0.52      0.52      0.52      2556
weighted avg       0.52      0.52      0.52      2556

Mean Average Precision: 0.5097935281942552
Area Under ROC Curve: 0.5187793427230047
Precision:  0.521505376344086
Recall:  0.45539906103286387
[LibLinear]Edge Operator: Hadamard
TP:  574
TN:  704
FP:  574
FN:  704
F1: 0.47320692497938993
[[704 574]
 [704 574]]
              precision    recall  f1-score   support

           0       0.50      0.55      0.52      1278
           1       0.50      0.45      0.47      1278

   micro avg       0.50      0.50      0.50      2556
   macro avg       0.50      0.50      0.50      2556
weighted avg       0.50 

### sum

In [25]:
for i in range(len(emb_list)):
    emb_list[i] = np.asfarray(emb_list[i],float)
node_embeddings_training = np.sum(np.asarray(emb_list[0:-2]),axis=0)
node_embeddings_testing = np.sum(np.asarray(emb_list[0:-1]),axis=0)

In [26]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  599
TN:  755
FP:  523
FN:  679
F1: 0.49916666666666676
[[755 523]
 [679 599]]
              precision    recall  f1-score   support

           0       0.53      0.59      0.56      1278
           1       0.53      0.47      0.50      1278

   micro avg       0.53      0.53      0.53      2556
   macro avg       0.53      0.53      0.53      2556
weighted avg       0.53      0.53      0.53      2556

Mean Average Precision: 0.5158740121457603
Area Under ROC Curve: 0.5297339593114241
Precision:  0.533868092691622
Recall:  0.46870109546165883
[LibLinear]Edge Operator: Hadamard
TP:  605
TN:  666
FP:  612
FN:  673
F1: 0.4849699398797595
[[666 612]
 [673 605]]
              precision    recall  f1-score   support

           0       0.50      0.52      0.51      1278
           1       0.50      0.47      0.48      1278

   micro avg       0.50      0.50      0.50      2556
   macro avg       0.50      0.50      0.50      2556
weighted avg       0.50 

### expdecay

In [31]:
exps = [np.exp(-i * 0.3) for i in range(1,21)]
node_embeddings_training = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-2],exps[:-1]):
    node_embeddings_training += e * c 
node_embeddings_testing = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-1],exps[:-1]):
    node_embeddings_testing += e * c 
print(len(emb_list))
print(len(exps))

In [32]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  604
TN:  746
FP:  532
FN:  674
F1: 0.500414250207125
[[746 532]
 [674 604]]
              precision    recall  f1-score   support

           0       0.53      0.58      0.55      1278
           1       0.53      0.47      0.50      1278

   micro avg       0.53      0.53      0.53      2556
   macro avg       0.53      0.53      0.53      2556
weighted avg       0.53      0.53      0.53      2556

Mean Average Precision: 0.5149771870660583
Area Under ROC Curve: 0.528169014084507
Precision:  0.5316901408450704
Recall:  0.4726134585289515
[LibLinear]Edge Operator: Hadamard
TP:  636
TN:  653
FP:  625
FN:  642
F1: 0.5009846396218983
[[653 625]
 [642 636]]
              precision    recall  f1-score   support

           0       0.50      0.51      0.51      1278
           1       0.50      0.50      0.50      1278

   micro avg       0.50      0.50      0.50      2556
   macro avg       0.50      0.50      0.50      2556
weighted avg       0.50    

### Amazon-Food - Equal Monthly
### load data

In [33]:
# load the training and testing graph
with open('../data/amazon_food/amazon_food_equal_monthly_dir.pkl', 'rb') as file:
    graphs = pickle.load(file)
    
graphs = graphs[-20:]
graph_train = graphs[-2]
graph_test = graphs[-1]

# parameters
num_edge_sample = 400

In [34]:
num_snapshots = len(graphs)
num_snapshots

20

In [35]:
emb_list = []
for i in range(num_snapshots):
    file = '../data/amazon_food/N2V_Embeddings/amazon_equal_month/amazon_equal_month_' + str(i) + '.npy'
    node_embedding = np.load(file)
    emb_list.append(node_embedding)
print(len(emb_list))

20


### independent / no combination

In [36]:
node_embeddings_training = emb_list[-3]
node_embeddings_testing = emb_list[-2]

In [37]:
for edge_operator in ['Average','Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  251
TN:  410
FP:  220
FN:  379
F1: 0.4559491371480472
[[410 220]
 [379 251]]
              precision    recall  f1-score   support

           0       0.52      0.65      0.58       630
           1       0.53      0.40      0.46       630

   micro avg       0.52      0.52      0.52      1260
   macro avg       0.53      0.52      0.52      1260
weighted avg       0.53      0.52      0.52      1260

Mean Average Precision: 0.5131112459137936
Area Under ROC Curve: 0.5246031746031745
Precision:  0.5329087048832272
Recall:  0.3984126984126984
[LibLinear]Edge Operator: Hadamard
TP:  303
TN:  296
FP:  334
FN:  327
F1: 0.478295185477506
[[296 334]
 [327 303]]
              precision    recall  f1-score   support

           0       0.48      0.47      0.47       630
           1       0.48      0.48      0.48       630

   micro avg       0.48      0.48      0.48      1260
   macro avg       0.48      0.48      0.48      1260
weighted avg       0.48   

### sum

In [38]:
for i in range(len(emb_list)):
    emb_list[i] = np.asfarray(emb_list[i],float)
node_embeddings_training = np.sum(np.asarray(emb_list[0:-2]),axis=0)
node_embeddings_testing = np.sum(np.asarray(emb_list[0:-1]),axis=0)

In [39]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  286
TN:  377
FP:  253
FN:  344
F1: 0.4893071000855432
[[377 253]
 [344 286]]
              precision    recall  f1-score   support

           0       0.52      0.60      0.56       630
           1       0.53      0.45      0.49       630

   micro avg       0.53      0.53      0.53      1260
   macro avg       0.53      0.53      0.52      1260
weighted avg       0.53      0.53      0.52      1260

Mean Average Precision: 0.5138969873663751
Area Under ROC Curve: 0.5261904761904762
Precision:  0.5306122448979592
Recall:  0.45396825396825397
[LibLinear]Edge Operator: Hadamard
TP:  284
TN:  353
FP:  277
FN:  346
F1: 0.4769101595298068
[[353 277]
 [346 284]]
              precision    recall  f1-score   support

           0       0.51      0.56      0.53       630
           1       0.51      0.45      0.48       630

   micro avg       0.51      0.51      0.51      1260
   macro avg       0.51      0.51      0.50      1260
weighted avg       0.51 

### expdecay

In [44]:
exps = [np.exp(-i * 0.3) for i in range(1,21)]
node_embeddings_training = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-2],exps[:-1]):
    node_embeddings_training += e * c 
node_embeddings_testing = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-1],exps[:-1]):
    node_embeddings_testing += e * c 
print(len(emb_list))
print(len(exps))

20
20


In [45]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  291
TN:  402
FP:  228
FN:  339
F1: 0.5065274151436031
[[402 228]
 [339 291]]
              precision    recall  f1-score   support

           0       0.54      0.64      0.59       630
           1       0.56      0.46      0.51       630

   micro avg       0.55      0.55      0.55      1260
   macro avg       0.55      0.55      0.55      1260
weighted avg       0.55      0.55      0.55      1260

Mean Average Precision: 0.5280346820809249
Area Under ROC Curve: 0.55
Precision:  0.5606936416184971
Recall:  0.46190476190476193
[LibLinear]Edge Operator: Hadamard
TP:  320
TN:  318
FP:  312
FN:  310
F1: 0.5071315372424723
[[318 312]
 [310 320]]
              precision    recall  f1-score   support

           0       0.51      0.50      0.51       630
           1       0.51      0.51      0.51       630

   micro avg       0.51      0.51      0.51      1260
   macro avg       0.51      0.51      0.51      1260
weighted avg       0.51      0.51     

### Amazon-Food - Equal Weekly
### load data

In [46]:
# load the training and testing graph
with open('../data/amazon_food/amazon_food_equal_weekly_dir.pkl', 'rb') as file:
    graphs = pickle.load(file)
    
graphs = graphs[-20:]
graph_train = graphs[-2]
graph_test = graphs[-1]

# parameters
num_edge_sample = 400

In [47]:
num_snapshots = len(graphs)
num_snapshots

20

In [48]:
emb_list = []
for i in range(num_snapshots):
    file = '../data/amazon_food/N2V_Embeddings/amazon_equal_week/amazon_equal_week_' + str(i) + '.npy'
    node_embedding = np.load(file)
    emb_list.append(node_embedding)
print(len(emb_list))

20


### independent / no combination

In [49]:
node_embeddings_training = emb_list[-3]
node_embeddings_testing = emb_list[-2]

In [50]:
for edge_operator in ['Average','Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  142
TN:  206
FP:  114
FN:  178
F1: 0.4930555555555555
[[206 114]
 [178 142]]
              precision    recall  f1-score   support

           0       0.54      0.64      0.59       320
           1       0.55      0.44      0.49       320

   micro avg       0.54      0.54      0.54       640
   macro avg       0.55      0.54      0.54       640
weighted avg       0.55      0.54      0.54       640

Mean Average Precision: 0.524267578125
Area Under ROC Curve: 0.5437500000000001
Precision:  0.5546875
Recall:  0.44375
[LibLinear]Edge Operator: Hadamard
TP:  177
TN:  165
FP:  155
FN:  143
F1: 0.5429447852760735
[[165 155]
 [143 177]]
              precision    recall  f1-score   support

           0       0.54      0.52      0.53       320
           1       0.53      0.55      0.54       320

   micro avg       0.53      0.53      0.53       640
   macro avg       0.53      0.53      0.53       640
weighted avg       0.53      0.53      0.53      

### sum

In [51]:
for i in range(len(emb_list)):
    emb_list[i] = np.asfarray(emb_list[i],float)
node_embeddings_training = np.sum(np.asarray(emb_list[0:-2]),axis=0)
node_embeddings_testing = np.sum(np.asarray(emb_list[0:-1]),axis=0)

In [52]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  143
TN:  201
FP:  119
FN:  177
F1: 0.49140893470790376
[[201 119]
 [177 143]]
              precision    recall  f1-score   support

           0       0.53      0.63      0.58       320
           1       0.55      0.45      0.49       320

   micro avg       0.54      0.54      0.54       640
   macro avg       0.54      0.54      0.53       640
weighted avg       0.54      0.54      0.53       640

Mean Average Precision: 0.5204675572519084
Area Under ROC Curve: 0.5375
Precision:  0.5458015267175572
Recall:  0.446875
[LibLinear]Edge Operator: Hadamard
TP:  159
TN:  190
FP:  130
FN:  161
F1: 0.5221674876847291
[[190 130]
 [161 159]]
              precision    recall  f1-score   support

           0       0.54      0.59      0.57       320
           1       0.55      0.50      0.52       320

   micro avg       0.55      0.55      0.55       640
   macro avg       0.55      0.55      0.54       640
weighted avg       0.55      0.55      0.54   

### expdecay

In [58]:
exps = [np.exp(-i * 0.3) for i in range(1,21)]
node_embeddings_training = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-2],exps[:-1]):
    node_embeddings_training += e * c 
node_embeddings_testing = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-1],exps[:-1]):
    node_embeddings_testing += e * c 
print(len(exps))
print(len(emb_list))

20
20


In [59]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  144
TN:  197
FP:  123
FN:  176
F1: 0.4906303236797275
[[197 123]
 [176 144]]
              precision    recall  f1-score   support

           0       0.53      0.62      0.57       320
           1       0.54      0.45      0.49       320

   micro avg       0.53      0.53      0.53       640
   macro avg       0.53      0.53      0.53       640
weighted avg       0.53      0.53      0.53       640

Mean Average Precision: 0.5176966292134831
Area Under ROC Curve: 0.5328124999999999
Precision:  0.5393258426966292
Recall:  0.45
[LibLinear]Edge Operator: Hadamard
TP:  161
TN:  150
FP:  170
FN:  159
F1: 0.4946236559139785
[[150 170]
 [159 161]]
              precision    recall  f1-score   support

           0       0.49      0.47      0.48       320
           1       0.49      0.50      0.49       320

   micro avg       0.49      0.49      0.49       640
   macro avg       0.49      0.49      0.49       640
weighted avg       0.49      0.49      